Foreword: You need at least 75 GB of RAM available, so if your runtime only gives you e.g. 12.5 system RAM, please reconnect until you're awarded 85 GB

# Necessary Imports

In [ ]:
# unfortunately, we must reinstall deepcell every runtime
!pip install deepcell -q

In [ ]:
from deepcell.applications import Mesmer
from deepcell.utils.plot_utils import create_rgb_image
from deepcell.utils.plot_utils import make_outline_overlay

In [ ]:
from google.colab import drive
import skimage.io as io
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# you should have the ome tiff in your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# replace the file path with your image's, you can find this by right-clicking
# on the file in the left panel and copy its path
file_path = "/content/drive/MyDrive/Colab Files/RAM13_003-Brain_131199-2.ome.tif"

# Optional: check GPU is accessible

In [ ]:
!nvidia-smi

In [ ]:
# let's check the GPU is on (it should output '/device:GPU:0').
# If it isn't check that you have enough compute units left or
# click Runtime -> Change runtime type -> select GPU A100 or V100
tf.test.gpu_device_name()

# Import and Pre-Process Image and Create RGB of DAPI

In [ ]:
image = io.imread(file_path)

# we must add an extra axis to fit the parameter requirements
# the extra axis is the batch, but we won't care about that
image = np.expand_dims(image, axis=0)
image.shape

# optional: visualize dapi

In [ ]:
# at least create the DAPI image. The other channels are commented
# out for efficiency, but you may display them just as you display
# rgb_image1 if you so choose.

# rgb_image8 = create_rgb_image(image[:,:,:,7:], channel_colors=['blue'])
# rgb_image7 = create_rgb_image(image[:,:,:,6:7], channel_colors=['blue'])
# rgb_image6 = create_rgb_image(image[:,:,:,5:6], channel_colors=['blue'])
# rgb_image5 = create_rgb_image(image[:,:,:,4:5], channel_colors=['blue'])
# rgb_image4 = create_rgb_image(image[:,:,:,3:4], channel_colors=['blue'])
# rgb_image3 = create_rgb_image(image[:,:,:,2:3], channel_colors=['blue'])
# rgb_image2 = create_rgb_image(image[:,:,:,1:2], channel_colors=['green'])
rgb_image1 = create_rgb_image(image[:,:,:,0:1], channel_colors=['blue'])

In [ ]:
# Let's look at the DAPI
# select channel index for displaying
idx = 0

# plot the data
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
ax.imshow(rgb_image1[idx, ...])

ax.set_title('DAPI')

plt.show()

# Predict

In [ ]:
# perform nuclear segmentation
# this usually takes 14 minutes and will use up 75 GB of RAM

app = Mesmer()
labeled_image = app.predict(image[:,:,:,0:2], image_mpp=app.model_mpp, compartment='nuclear')

# Optional: Make RGB Overlay and Visualize Overlay

In [ ]:
# make the RGB overlay for optional visualization
overlay_data_nuc_big = make_outline_overlay(rgb_data=rgb_image1, predictions=labeled_image)

In [ ]:
# optional
# display the overlay
# select index for displaying
idx = 0

# plot the data
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
ax.imshow(overlay_data_nuc_big[idx, ...])
ax.set_title('Nuclear Predictions')
plt.show()

# Save Mask as Tiff to Google Drive

In [ ]:
# we don't need the extra batch and channel dimensions for the mask
squeezed_array_big = np.squeeze(labeled_image[0,:,:,:])

In [ ]:
# optional: if you want to view the mask
binary_mask_big = (squeezed_array_big > 0).astype(int)

plt.imshow(binary_mask_big, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

In [ ]:
# optional: if you want to see how many cells detected
max_value = np.max(squeezed_array_big)
print(max_value)

In [ ]:
# save the mask
# import skimage.io as io
# io.imsave("ome_mask_2D.tif", squeezed_array_big, plugin="tifffile", bigtiff=True)

np.save('mask.npy', squeezed_array_big)

# upload to Google Drive
# !cp ome_mask_2D.tif "/content/drive/My Drive/Colab Files"
!cp mask.npy "/content/drive/My Drive/Colab Files"

If You Want to Generate Cytoplasms in Python (Step 2 Method 2). Do the following:

In [ ]:
# create pseudo-cytoplasms using constant expansion factor
from skimage.segmentation import expand_labels
expanded = expand_labels(mask, distance=10)
np.save('data/expanded.npy', expanded)
# save cytoplasm mask tiff
# io.imsave("ome_cytoplasms_2D.tif", expanded, plugin="tifffile", bigtiff=True)

# upload to Google Drive
# !cp ome_cytoplasms_2D.tif "/content/drive/My Drive/Colab Files"
!cp expanded.npy "/content/drive/My Drive/Colab Files"

In [ ]:
# print(np.array_equal(np.sort(np.unique(mask)), np.sort(np.unique(expanded))))

True
